In [60]:
import gzip
import json
import pandas as pd
import numpy as np
import re
import ast
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from multiprocessing import Pool
from transformers import pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow.parquet as pq
from memory_profiler import memory_usage
import os
import time

### Before starting our ETL, we did a prior processing of the original databases with the tool "codebeautyfy.org/json-fixer"
### To obtain valid json files.

In [61]:
# Constante para la ruta de nuestros archivos [0] = Games [1] = reviews [2] = items

FILE_NAMES = ['.\\Datasets\\steam_games.json.gz','.\\Datasets\\user_reviews_fixed.json.gz','.\\Datasets\\users_items_fixed.json.gz']

In [62]:
def file_to_dataframe(file):

    """
    Create a pandas dataframe from a JSON file.

    Parameters
    ----------
    file : str
        The path or URL of the JSON file.

    Returns
    -------
    file_data_frame : pd.DataFrame
        The dataframe created from the JSON file.

    Raises
    ------
    ValueError
        If the JSON file is not valid or cannot be read.
    """
    try:
        file_data_frame = pd.read_json(file, compression='gzip',lines=True)
        return file_data_frame
    except ValueError:
        file_data_frame = pd.read_json(file, compression='gzip')
        return file_data_frame
    except:
        raise ValueError('Something went wrong, Dataframe not created')

In [63]:
# Cargamos los Json a cada una de las dataframes

steam_games_df = file_to_dataframe(FILE_NAMES[0])
user_reviews_df = file_to_dataframe(FILE_NAMES[1])
user_items_df = file_to_dataframe(FILE_NAMES[2])


Null Treatment for dataframes

In [64]:
steam_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [65]:
user_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [66]:
user_items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  int64 
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.4+ MB


In [67]:
def drop_empty_rows(dataframe):

    """
    Drop rows from a dataframe that contain only missing values.

    Parameters
    ----------
    dataframe : pd.DataFrame
        The dataframe to drop rows from.

    Returns
    -------
    pd.DataFrame
        The dataframe with empty rows dropped. The original dataframe is modified in place.
    """
    dataframe.dropna(how='all', inplace=True)
    return dataframe

steam_games_df = drop_empty_rows(steam_games_df)
user_reviews_df = drop_empty_rows(user_reviews_df)
user_items_df = drop_empty_rows(user_items_df)

### Removemos las columnas que no utilizaremos a futuro

In [68]:
# Se detecta que app_name y title son básicamente lo mismo, se decide dropear title porque tiene vacios y app_name no

def compare_name_title(steam_games_df):
    """
    Compare the app_name and title columns of a dataframe of Steam games.

    This function prints the number of null values in each column and displays
    a subset of the dataframe where the app_name and title are different and
    not null.

    Parameters
    ----------
    steam_games_df : pd.DataFrame
        The dataframe of Steam games to compare.

    Returns
    -------
    None
        The function does not return anything. It only prints and displays data.
    """ 
    
    print(f"Nulos en title : {steam_games_df['title'].isnull().sum()}")
    print(f"Nullos en app_name: {steam_games_df['app_name'].isnull().sum()}")
    
    compare_appname_title = steam_games_df[(steam_games_df['app_name'] != steam_games_df['title']) & (~steam_games_df['app_name'].isnull()) & (~steam_games_df['title'].isnull())]
    display(compare_appname_title[['app_name','title']])
    
compare_name_title(steam_games_df)

Nulos en title : 2050
Nullos en app_name: 2


,app_name,title
88390,Sam & Max 101: Culture Shock,Sam &amp; Max 101: Culture Shock
88393,Sam & Max 102: Situation: Comedy,Sam &amp; Max 102: Situation: Comedy
88419,Command & Conquer: Red Alert 3,Command &amp; Conquer: Red Alert 3
88492,Heroes of Might & Magic V: Hammers of Fate,Heroes of Might &amp; Magic V: Hammers of Fate
88494,Heroes of Might & Magic V: Tribes of the East,Heroes of Might &amp; Magic V: Tribes of the East
...,...,...
120181,Sam & Max 105: Reality 2.0,Sam &amp; Max 105: Reality 2.0
120182,Sam & Max 104: Abe Lincoln Must Die!,Sam &amp; Max 104: Abe Lincoln Must Die!
120183,Sam & Max 106: Bright Side of the Moon,Sam &amp; Max 106: Bright Side of the Moon
120208,Making History: The Calm & the Storm,Making History: The Calm &amp; the Storm


In [69]:
# Dropeamos columnas que no usaremos a futuro
# steam_Games | reviews_url: para ninguno de nuestros análisis, manejo o muestra de datos utilizaremos sus url 
#             | title :  verificamos que el title es lo mismo que el app_name pero con muchos vacíos así que decidimos dropearlos

# user_reviews | user_url: para ninguno de nuestros análisis, manejo o muestra de datos utilizaremos sus url
# user_items | user_url: para ninguno de nuestros análisis, manejo o muestra de datos utilizaremos sus url

# Limitación: Para fines del proyecto no se contempla pero podríamos mejorar la base de datos haciendo un webScrapping con las url

steam_games_df = steam_games_df.drop('reviews_url', axis=1)
steam_games_df = steam_games_df.drop('title', axis=1)

# Dropeamos los 2 nulos de app_name
steam_games_df = steam_games_df.dropna(subset=['app_name'])

user_items_df = user_items_df.drop('user_url',axis=1)
user_reviews_df = user_reviews_df.drop('user_url', axis=1)

## Chequeo de columnas numericas

In [70]:
# Steam | Price

not_numeric = []
def not_numeric_values(value):
    try : 
        num = float(value)
        return None
    except ValueError:
        not_numeric.append(value)
    except TypeError:
        not_numeric.append(-1)


def convert_price(price):
    if price in ['Free Demo','Free to Play','Free To Play','Play for Free!','Free', 'Install Now', 'Play WARMACHINE: Tactics Demo', 
                 'Free Mod','Install Theme','Third-party','Play Now','Free HITMAN™ Holiday Pack','Install Now','Play the Demo','Free to Try',
                 'Free Movie','Free to Use']:
        return 0
    elif price == 'Starting at $499.00':
        return 499.00
    elif price == 'Starting at $449.00':
        return 449.00
    else:
        return price

# We apply the price conversion to all the data of type String
steam_games_df.loc[:,'price'] = steam_games_df['price'].apply(convert_price)

# Function used to find the non-numeric data
steam_games_df['price'].apply(not_numeric_values)        
print(not_numeric) 

# Fill the empty data with the mean of the column
# Decided to use the mean because it is less affected by outliers
# Decided not to drop these data to avoid affecting future functions based on the price of games

steam_games_df['price'] = steam_games_df['price'].fillna(steam_games_df['price'].median())
steam_games_df.info()
steam_games_df

[]
<class 'pandas.core.frame.DataFrame'>
Index: 32133 entries, 88310 to 120444
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28851 non-null  object 
 2   app_name      32133 non-null  object 
 3   url           32133 non-null  object 
 4   release_date  30067 non-null  object 
 5   tags          31971 non-null  object 
 6   specs         31464 non-null  object 
 7   price         32133 non-null  float64
 8   early_access  32133 non-null  float64
 9   id            32132 non-null  float64
 10  developer     28836 non-null  object 
dtypes: float64(3), object(8)
memory usage: 2.9+ MB


,publisher,genres,app_name,url,release_date,tags,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [71]:
steam_games_df['release_date'] = pd.to_datetime(steam_games_df['release_date'], format='%Y-%m-%d', errors='coerce')
steam_games_df['year_of_release'] = steam_games_df['release_date'].dt.year
steam_games_df

## No voy a tocar los nulos en caso tal les pondré 2016 que es la media 
## ahorita no quiero poner la media porque modificara las horas jugadas por año

,publisher,genres,app_name,url,release_date,tags,specs,price,early_access,id,developer,year_of_release
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,0.0,761140.0,Kotoshiro,2018.0
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,643980.0,Secret Level SRL,2018.0
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",0.00,0.0,670290.0,Poolians.com,2017.0
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,0.0,767400.0,彼岸领域,2017.0
88314,None,None,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaT,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS""",2018.0
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada,2018.0
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich,2018.0
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns",2017.0


## FEATURING ENGINEERING

#### Reviews Sentiment Analysis

In [72]:
# nltk.download("vader_lexicon")



In [73]:
def get_sentiment(review):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(review)
    return sentiment['compound']

def get_user_sentiment(reviews):
    sentiments = []
    for review in reviews:
        sentiment = get_sentiment(review['review'])
        # sentiments.append(sentiment)
        dict_review = [{'Posted: ':review['posted'], 'Last_Edited':review['last_edited'],'item_id':review['item_id'], 'recommend':review['recommend'] , 'sentiment':sentiment}]
        sentiments.append(dict_review)
    # avg_sentiment = sum(sentiments)  # maybe solo la suma
    return sentiments

def add_sentiment_column(df):
    df['sentiment'] = user_reviews_df['reviews'].apply(get_user_sentiment)

def user_review_explode(review_df):
    review_df = review_df[['user_id','sentiment']]
    review_df = review_df.explode('sentiment')
    review_df = review_df.explode('sentiment')
    hold_user = review_df[['user_id']]
    hold_user = hold_user.reset_index(drop=True)
    sentiment_exploded = pd.json_normalize(review_df['sentiment'])
    output_df = pd.concat([hold_user,sentiment_exploded], axis=1, join='inner')
    return output_df

In [74]:
# user_reviews_df
add_sentiment_column(user_reviews_df)
user_reviews_df = user_review_explode(user_reviews_df)
user_reviews_df


,user_id,Posted:,Last_Edited,item_id,recommend,sentiment
0,76561197970982479,"Posted November 5, 2011.",,1250,True,0.8481
1,76561197970982479,"Posted July 15, 2011.",,22200,True,0.2263
2,76561197970982479,"Posted April 21, 2011.",,43110,True,0.9117
3,js41637,"Posted June 24, 2014.",,251610,True,0.9566
4,js41637,"Posted September 8, 2013.",,227300,True,0.9708
...,...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,,70,True,0.5574
59329,76561198312638244,Posted July 8.,,362890,True,0.9786
59330,LydiaMorley,Posted July 3.,,273110,True,0.7827
59331,LydiaMorley,Posted July 20.,,730,True,0.5106


### User_Items

In [75]:
user_items_df

,user_id,items_count,steam_id,items
0,76561197970982479,277,76561197970982480,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864384,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712560,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445856,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099488,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...
88305,76561198323066619,22,76561198323066624,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700688,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759264,[]
88308,76561198329548331,7,76561198329548336,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [76]:
user_items_df.drop('steam_id', axis=1)
user_items_df.drop('items_count', axis=1)
user_items_df.to_parquet('borrar_user_items_df.parquet')

## PARQUETS

#### Query 1

In [77]:
query1_parquet = steam_games_df[['developer','price','release_date']]
query1_parquet.to_parquet('query1.parquet')

#### Query 2

In [78]:
query_2_steam_parquet = steam_games_df[['price','id']]
query_2_steam_parquet.to_parquet('query_2_steam.parquet')

query_2_items_parquet = user_items_df[['user_id','items']]
query_2_items_parquet.to_parquet('query_2_items.parquet')

query_2_reviews_parquet = user_reviews_df[['user_id','recommend']]
query_2_reviews_parquet.to_parquet('query_2_reviews.parquet')

In [79]:
query_2_items_parquet

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...
88305,76561198323066619,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,[]
88308,76561198329548331,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [89]:
# userid = '76561197970982479'
# userid = 'erickoiv'
userid = 'jjas01'
# userid = '76561198079680944'

def read_parquets ():
    query_2_steam = pd.read_parquet('query_2_steam.parquet')
    query_2_reviews = pd.read_parquet('query_2_reviews.parquet')
    return query_2_steam, query_2_reviews

# Cantidad de dinero gastado por el usuario

def load_parquet_in_batches(file_path,user_id):
    """Loads a Parquet file in batches.

    Args:
    file_path: The path to the Parquet file.
    chunk_size: The size of each batch.

    Returns:
    A list of Pandas DataFrames.
    """
    parquet_file = pq.ParquetFile(file_path)
    dataframes = []
    for batch in parquet_file.iter_batches(batch_size=5000):
        dataframes = batch.to_pandas()
        result = get_items_names(dataframes,user_id)
        if result is not None :
            return result
    return result

 
def get_items_names(items_dataframe,user_id):
    items_dataframe = items_dataframe.set_index("user_id")    
    if user_id in items_dataframe.index:
        found_data_frame = items_dataframe.loc[user_id]

        if not found_data_frame.empty:
            return found_data_frame
    else:
        return None
    return None

def get_items_id(row_df):
    items= []
    user_items = row_df.iloc[0]

    for item in user_items:
        items.append(item['item_id'])
    return items

def get_waste(items_list):
    waste = 0
    prices = []
    for item in items_list:
        intintem=int(item)
        price = query_2_steam.query("id == @intintem")
        # prices.append((price['price'].values))

        if len(price['price'].values) > 0:
            waste = waste+price['price'].values[0]
            prices.append((price['price'].values))
    return waste


def percent_reviews(percent_user_id):
    try:
        recommends = query_2_reviews.query("user_id == @percent_user_id")
        recommends_true = query_2_reviews.query("(user_id == @percent_user_id) & (recommend == True)")
        items_amount = len(recommends)
        percent = (len(recommends_true) * 100) / items_amount
        return percent, items_amount
    except ZeroDivisionError:
        return 0,0
    
    ##################################
api_df = pd.DataFrame(columns=['user_id','total_waste','reviews_percent','total_reviews'])
def api_parquet(userid, total_waste, reviews_items_percent,total_reviews,api_df):
    api_df.loc[len(api_df.index)] = [userid,total_waste,reviews_items_percent,total_reviews]
     
    return (api_df)

def get_items_api(api_user_items):
    items = []
    for item in api_user_items:
            items.append(item['item_id'])
    return items

    

# Load small Parquets
query_2_steam, query_2_reviews = read_parquets()

x=0
for user in query_2_items_parquet.iterrows():

    # print(user)
    this_user_data = user[1]
    this_user = this_user_data['user_id']
    this_user_items = this_user_data['items']
    this_user_items =get_items_api(this_user_data['items'])

    # query_2_items = load_parquet_in_batches('query_2_items.parquet',this_user)
    # items_list = get_items_id(query_2_items) 
    # print(items_list)

    total_waste = get_waste(this_user_items)
    reviews_items_percent, total_reviews = percent_reviews(this_user)
    api_df = api_parquet(this_user,total_waste,reviews_items_percent,total_reviews,api_df)

    #break
    # print(this_user['id'])

# Load the Parquet file in batches
# query_2_items = load_parquet_in_batches('query_2_items.parquet',userid)

# items_list = get_items_id(query_2_items)  

# total_waste = get_waste(items_list)
# reviews_items_percent, total_reviews = percent_reviews(userid)

# api_df = api_parquet(userid,total_waste,reviews_items_percent,total_reviews,api_df)
# json_output = {userid: { "spent_money":total_waste, "recommendation %":reviews_items_percent, "reviews_amount":total_reviews}}
# json_output
api_df.to_parquet('api_query2.parquet')
# api_df


KeyboardInterrupt: 

In [ ]:
x = []
y = {}
g = {'id':'asldk'}
z = x + 1 + y - g

#### Query 3

userforgenre(genero: str)

Usuario con más horas jugadas para un género
dado el género
lista de acumulación de horas jugadas por año de lanzamiento

Usuario con más horas jugadas para género X : asldkjalksd 
Horas Jugadas: 2013: Horas 200
               2014: Horas 50
               2015: Horas 20

steam_games_df = genres | id
users_items_df = user_id | items

De la lista de usuarios buscar la ID de los juegos y la Hora Jugada de cada Juego podemos usar la misma función de antes

In [133]:
steam_games_df
query3_steam_df = steam_games_df[['genres','id','year_of_release']]
query3_steam_df

user_items_df
query3_user_items_df = user_items_df[['user_id','items']]
query3_user_items_df
# query3_steam_df

,genres,id,year_of_release
88310,"[Action, Casual, Indie, Simulation, Strategy]",761140.0,2018.0
88311,"[Free to Play, Indie, RPG, Strategy]",643980.0,2018.0
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",670290.0,2017.0
88313,"[Action, Adventure, Casual]",767400.0,2017.0
88314,None,773570.0,NaN
...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",773640.0,2018.0
120441,"[Casual, Indie, Strategy]",733530.0,2018.0
120442,"[Indie, Racing, Simulation]",610660.0,2018.0
120443,"[Casual, Indie]",658870.0,2017.0


In [91]:
query3_exploded_genre = query3_steam_df.explode('genres')
query3_exploded_genre

genre_grouped = query3_exploded_genre.groupby(['genres'])

genre_grouped = pd.DataFrame({'genres': genre_grouped['genres'].first(),
                            'id_list': genre_grouped['id'].apply(list)})

genre_grouped = genre_grouped.reset_index(drop=True)
genre_grouped['id_list'] = genre_grouped['id_list'].apply(set)
genre_grouped
# query3_steam_df

# genre_grouped.to_parquet('genre_grouped.parquet')

,genres,id_list
0,Accounting,"{555810.0, 732710.0, 623590.0, 620040.0, 41134..."
1,Action,"{327680.0, 655360.0, 32770.0, 524290.0, 393220..."
2,Adventure,"{327680.0, 393220.0, 720900.0, 622620.0, 45879..."
3,Animation &amp; Modeling,"{747520.0, 697860.0, 321540.0, 620040.0, 36302..."
4,Audio Production,"{620040.0, 497160.0, 281100.0, 281102.0, 28110..."
5,Casual,"{294912.0, 327680.0, 294914.0, 294915.0, 29491..."
6,Design &amp; Illustration,"{747520.0, 600070.0, 692240.0, 755743.0, 34820..."
7,Early Access,"{311310.0, 622610.0, 466980.0, 573490.0, 69637..."
8,Education,"{747520.0, 620040.0, 468510.0, 267810.0, 36919..."
9,Free to Play,"{720900.0, 458760.0, 327690.0, 565261.0, 40143..."


In [285]:
for row in genre_grouped['id_list']:
    print (len(row))

7
11319
8243
183
93
8282
460
1462
125
2031
15857
1108
77
5479
1083
6699
105
1257
6957
340
116
268


In [130]:
# df_grouped = developers_year.groupby(developers_year['release_date'].dt.year)['price'].agg(['sum', 'count'])

genre = 'Action'
genres_all = ['Accounting', 'Action', 'Adventure', 'Animation &amp; Modeling', 'Audio Production', 'Casual', 'Design &amp; Illustration', 'Early Access', 'Education', 'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing']
# genres_all = ['Action']
Global_Max_User_id = ''
Global_Max_Time_Played = 0
api_most_played_genre = pd.DataFrame(columns=['genre','user_id','total_played_time'])

def read_parquets ():
    genre_grouped_steam = pd.read_parquet('genre_grouped.parquet')
    return genre_grouped_steam

def load_parquet_in_batches(file_path,Global_Max_Time_Played, Global_Max_User_id,specific_genre_steam):
    """Loads a Parquet file in batches.

    Args:
    file_path: The path to the Parquet file.
    chunk_size: The size of each batch.

    Returns:
    A list of Pandas DataFrames.
    """
    parquet_file = pq.ParquetFile(file_path)
    dataframes = []
    for batch in parquet_file.iter_batches(batch_size=5000):
        dataframes = batch.to_pandas()
        Global_Max_Time_Played, Global_Max_User_id= check_global_user(dataframes,Global_Max_Time_Played, Global_Max_User_id,specific_genre_steam)
    return Global_Max_Time_Played, Global_Max_User_id

def get_frame_genre(genre_grouped_steam,genre):  
    genre_grouped_steam = genre_grouped_steam.set_index("genres")  
    if genre in genre_grouped_steam.index:
        found_data_frame = genre_grouped_steam.loc[genre]  # esto lo debería hacer una sola vez afuera
        # print('LEN ', len(found_data_frame['id_list']))
        return found_data_frame
    else:
        return None
    

def genre_check(genre_grouped_steam,item_id) :      
        return float(item_id) in genre_grouped_steam['id_list']


def check_global_user (batch_user_items,Genre_Max_Player_Time, Genre_Max_Player,specific_genre_steam):
    for row in batch_user_items.iterrows() :
        data_row = row[1]
        user_time_played = 0
        user_id_row = data_row['user_id']
        items_row = tuple(data_row['items'])

        # print(f'ROW:{row}')
        
        for item in items_row:
            # print(f'ITEM:{item}')
            if float(item['item_id']) in specific_genre_steam['id_list']: #aqui habia el genre viejo genre_grouped_steam
                # print(f'ENTRA IF:')
                user_time_played += float(item['playtime_forever'])
                
        if user_time_played > Genre_Max_Player_Time:
            # print(f'PLAYED_TIME: {user_time_played}')
            # print(f'USER: {user_id_row}')
            Genre_Max_Player_Time = user_time_played
            Genre_Max_Player = user_id_row
    return Genre_Max_Player_Time, Genre_Max_Player
        
genre_grouped_steam = read_parquets()

for genre in genres_all:
    # print(f'Tipo_Genero: {type(genre)}')
    # print(f'Genero_Name: {genre}')
    specific_genre_steam = get_frame_genre(genre_grouped_steam,genre)
    # print(f'list_grupo: {specific_genre_steam}')
    Genre_Max_Player = ''
    Genre_Max_Player_Time = 0
    print(specific_genre_steam)
    if specific_genre_steam is not None: Genre_Max_Player_Time, Genre_Max_Player = load_parquet_in_batches('query_2_items.parquet',Genre_Max_Player_Time, Genre_Max_Player,specific_genre_steam)
    # print('pasa el if?')
    api_most_played_genre.loc[len(api_most_played_genre.index)] = [genre,Genre_Max_Player,Genre_Max_Player_Time]
    # print('hace la agregacion?')

# print('sale del for?')
# print(Global_Max_Time_Played)
# print(Global_Max_User_id)

api_most_played_genre.to_parquet('api_most_played_genre.parquet')
api_most_played_genre



id_list    [555810.0, 732710.0, 623590.0, 620040.0, 41134...
Name: Accounting, dtype: object
id_list    [327680.0, 655360.0, 32770.0, 524290.0, 393220...
Name: Action, dtype: object
id_list    [327680.0, 393220.0, 720900.0, 622620.0, 45879...
Name: Adventure, dtype: object
id_list    [747520.0, 697860.0, 321540.0, 620040.0, 36302...
Name: Animation &amp; Modeling, dtype: object
id_list    [620040.0, 497160.0, 281100.0, 281102.0, 28110...
Name: Audio Production, dtype: object
id_list    [294912.0, 327680.0, 294914.0, 294915.0, 29491...
Name: Casual, dtype: object
id_list    [747520.0, 600070.0, 692240.0, 755743.0, 34820...
Name: Design &amp; Illustration, dtype: object
id_list    [311310.0, 622610.0, 466980.0, 573490.0, 69637...
Name: Early Access, dtype: object
id_list    [747520.0, 620040.0, 468510.0, 267810.0, 36919...
Name: Education, dtype: object
id_list    [720900.0, 458760.0, 327690.0, 565261.0, 40143...
Name: Free to Play, dtype: object
id_list    [327680.0, 655360.0, 360450.0,

,genre,user_id,total_played_time
0,Accounting,,0.0
1,Action,Sp3ctre,1699307.0
2,Adventure,REBAS_AS_F-T,2191551.0
3,Animation &amp; Modeling,ScottyG555,168314.0
4,Audio Production,Lickidactyl,109916.0
5,Casual,REBAS_AS_F-T,1224933.0
6,Design &amp; Illustration,ScottyG555,168314.0
7,Early Access,76561197978756659,316969.0
8,Education,76561198059330972,65427.0
9,Free to Play,idonothack,808241.0


In [288]:
#### HORAS JUGADAS POR AÑO POR GENERO
#### GENERO | 2012 | 2013 | 2014

## Ya yo tengo la lista de juegos de cada género
## de tengo que recorrer esa lista e ir sumandole al año del género en que salio ese juego
## lo que cada usuario jugó de ese juego


# genres_all2 = ['Accounting', 'Action', 'Adventure', 'Animation &amp; Modeling', 'Audio Production', 'Casual', 'Design &amp; Illustration', 'Early Access', 'Education', 'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing']

# year_genre_grouped = genre_grouped
# players_df = query3_user_items_df

# year_genre_grouped

# max_year = query3_steam_df['year_of_release'].max()
# print(max_year)

# games_df = query3_steam_df #lista de jugadores
# years_list = list(range(1970,2022))
# # print(years_list)
# years_df = pd.DataFrame()
# for year in years_list:
#     years_df[year]= year

# year_genre_grouped
# years_df 

# grouped_with_year_steam = pd.concat([year_genre_grouped, years_df], axis=1)
# grouped_with_year_steam = grouped_with_year_steam.fillna(0)
# # grouped_with_year_steam

# # genre_test = 'Action'
# # recorrer para cada jugador de players_df qquien
# def get_frame_genre2(genre_grouped_steam,genre):  
#     genre_grouped_steam = genre_grouped_steam.set_index("genres")  
#     if genre in genre_grouped_steam.index:
#         print(f"Entra con {genre}")
#         found_data_frame = genre_grouped_steam.loc[genre]  # esto lo debería hacer una sola vez afuera
#         # print('LEN ', len(found_data_frame['id_list']))
#         return found_data_frame
#     else:
#         return None


# # print(f"SPECIFIC_CARAJO {specific_genre_steam2}")

# def check_game_year(games_df,game_id):
#      #Buscar año del juego
#      #Indexar por id games_df
#      #Devolver el año
#     game_year = 0
#     games_df = games_df.set_index("id")  
#     # print(f'game_id_type:{type(game_id)}')
#     # print(f'game_index_type:{type(games_df.index)}')
#     # print("llega aqui 1")
#     if float(game_id) in games_df.index:
#         # print("llega aqui 2")
#         row_game = games_df.loc[float(game_id)]  # Encuentra la fila donde está la id
#         # print(f'ROW_GAME: {row_game}')
#         # print("llega aqui 3")
#         game_year = row_game['year_of_release']

#     return game_year
# def sum_time_to_year(grouped_with_year_steam,game_year,played_time,genre_test):
#     # print("entra sum?")
#     # row = grouped_with_year_steam.loc[grouped_with_year_steam['genres'] == genre_test]
#     # print(f"RRRROOW: {row}")
#     # grouped_with_year_steam = grouped_with_year_steam.set_index("genres")
#     # print("sum 2 ?")
#     grouped_with_year_steam.at[genre_test,game_year] += played_time

#     return grouped_with_year_steam

# grouped_with_year_steam = grouped_with_year_steam.set_index("genres")    
# x = 0
# start_time = time.time()
# for genre_test in genres_all2:
#     specific_genre_steam2 = get_frame_genre2(genre_grouped_steam,genre_test)
#     print(genre_test)
#     for row in players_df.iterrows():
#         x = x+1
#         if x == 10000:
#             end_time = time.time()
#             print(f"tiempo + {end_time - start_time}")
#         data_row = row[1]
#         user_id_row = data_row['user_id']
#         items_row = tuple(data_row['items'])
#         for item in items_row:
#             if float(item['item_id']) in specific_genre_steam2['id_list']: # Que hacemos si el juego del tipo está en la lista de genero
#             # Buscar año del juego
#                 game_year = check_game_year(games_df,item['item_id'])

#                 # Ahora que tenemos el año del jueego tambien buscamos las horas que jugó
#                 played_time = item['playtime_forever']
#                 #Ahora que tenemos el año y el tiempo jugado debemos sumar este tiempo en el año de la otra DF (grouped_with_years DF)

#                 if not np.isnan(game_year):
#                     grouped_with_year_steam = sum_time_to_year(grouped_with_year_steam,game_year,played_time,genre_test)
#     break


    #Tengo usuario e items Chequear para cada uno de estos items si es del tipo del género con el q trabajamos
    # break

# grouped_with_year_steam[['genres']]
# players_df
# specific_genre_steam
# print(games_df[games_df['id'] == 10])
# games_df


2021.0
Entra con Accounting
Accounting
tiempo + 7.3409423828125


KeyboardInterrupt: 

In [303]:

# grouped_with_year_steam[['genres']]
# players_df
# specific_genre_steam
# print(games_df[games_df['id'] == 10])
# grouped_with_year_steam

# mask = grouped_with_year_steam[[1999]]
# mask
genres_all2 = ['Accounting', 'Action', 'Adventure', 'Animation &amp; Modeling', 'Audio Production', 'Casual', 'Design &amp; Illustration', 'Early Access', 'Education', 'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing', 'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports', 'Strategy', 'Utilities', 'Video Production', 'Web Publishing']

players_df = query3_user_items_df
games_df = query3_steam_df #lista de juegos

# def played_time_per_game(id_column):
#     for id in id_column:
        # Recorrer todos los jugadores

# games_df['played_time'] = played_time_per_game(games_df[['id']])

players_df

def iterate_players(players_df):
    for items in players_df['items']:
        print(items)
iterate_players(players_df)


SyntaxError: expected ':' (614844218.py, line 23)